Lecture: AI I - Advanced 

Previous:
[**Chapter 2.3: Ensemble learning**](../03_ensemble.ipynb)

---

# Exercise 2.3: Ensemble learning

> Hint: When doing the exercises put your solution in the designated "Solution" section:
> ```python
> # Solution (put your code here)
> ```

## Task 1: Diabetes Regression with multiple Multi-Layer Perceptron

The diabetes dataset contains 442 samples with 10 baseline variables (age, sex, BMI, blood pressure, and 6 blood serum measurements). The target is a quantitative measure of disease progression one year after baseline.

**Tasks**
- Data Exploration & Understanding
- Data preparation
- Build Models: multiple Multi-Layer Perceptrons for regression (at least 3 with different architectures)
- Train the models 
- Construct an ensemble model from the individual models
- Evaluate the model performance using appropriate regression metrics (e.g. MSE, MAE) and Tensorboard


In [ ]:
# prerequisites (don't edit this block)
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
x = diabetes.data
y = diabetes.target

In [ ]:
# Solution (put your code here)


In [ ]:
# Test case (don't edit this block)
assert True

---

Lecture: AI I - Advanced 

Next: [**Chapter 3.1: PyTorch Lightning**](../../03_advanced/01_lightning.ipynb)